In [4]:
#AutoPowerTester 

import tkinter as tk
from tkinter import messagebox, ttk, filedialog
import pandas as pd
import pyvisa
import time
from datetime import datetime
import os
import random  # Added for pseudo measurements
import threading
import queue
import json

# ---------- Config file handling for model settings ----------
try:
    SCRIPT_DIR = os.path.dirname(__file__)
except NameError:
    SCRIPT_DIR = os.getcwd()  # __file__ is undefined in notebooks
CONFIG_FILE = os.path.join(SCRIPT_DIR, "config.json")

def load_config():
    # Default values if no config exists or loading fails
    default_voltage_map = {
        "F966": 5.8,
        "S931": 5.8,
        "S936": 4.2,
        "S938": 5.0,
        "S721": 5.8,
        "A166": 5.2
    }
    default_criteria = {
        "F966": {"PASS": 0.8, "F7A": 0.6},
        "S931": {"PASS": 0.8, "F7A": 0.6},   # PASS >0.8, F7A 0.6~0.8, F7P <0.6
        "S721": {"PASS": 0.75, "F7P": 0.6},  # PASS >0.75, F7P 0.6~0.75, F7A <0.6
        # Add other models as needed
    }
    default_gpib_address = {
        "F966": 6,
        "S931": 6,
        "S936": 6,
        "S938": 6,
        "S721": 6,
        "A166": 6
    }

    if not os.path.exists(CONFIG_FILE):
        return default_voltage_map, default_criteria, default_gpib_address

    try:
        with open(CONFIG_FILE, "r", encoding="utf-8") as f:
            data = json.load(f)
        mv = data.get("MODEL_VOLTAGE_MAP", default_voltage_map)
        mc = data.get("MODEL_CRITERIA", default_criteria)
        mga = data.get("MODEL_GPIB_ADDRESS", default_gpib_address)
        return mv, mc, mga
    except Exception:
        # If config is broken, fall back to defaults
        return default_voltage_map, default_criteria, default_gpib_address

def save_config():
    data = {
        "MODEL_VOLTAGE_MAP": MODEL_VOLTAGE_MAP,
        "MODEL_CRITERIA": MODEL_CRITERIA,
        "MODEL_GPIB_ADDRESS": MODEL_GPIB_ADDRESS
    }
    with open(CONFIG_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)

# --------- Model to Voltage Mapping / Criteria (loaded from config) ---------
MODEL_VOLTAGE_MAP, MODEL_CRITERIA, MODEL_GPIB_ADDRESS = load_config()

# Toggle to let users switch between real instrument readings and pseudo data
USE_PSEUDO_CURRENT = False
ACTIVE_MEASUREMENT = {"thread": None, "stop_event": None}

# log path
today_date = datetime.now().strftime("%m%d%y")
documents_folder = os.path.join(os.path.expanduser("~"), r"Documents\AutoPowerTester Log")
# Create folder if it doesn't exist
os.makedirs(documents_folder, exist_ok=True)
log_saved_path = os.path.join(documents_folder, f"AutoPowerTester_Log_{today_date}.xlsx")

# --------- Instrument Measurement Function ---------

def measure_current_and_get_avg(model_voltage, gpib_address=6):
    if USE_PSEUDO_CURRENT:
        total_samples = 30
        pseudo_values = [psuedo_current(model_voltage) for _ in range(total_samples)]
        if not pseudo_values:
            raise RuntimeError("Pseudo measurement did not generate samples.")
        window = pseudo_values[-5:] if len(pseudo_values) >= 5 else pseudo_values
        return sum(window) / len(window)

    resource_str = f'GPIB0::{gpib_address}::INSTR'
    rm = pyvisa.ResourceManager()
    inst = rm.open_resource(resource_str)
    inst.write('OUTP ON')
    inst.write(f"VOLT {model_voltage}")

    iteration = 0
    current_values = []

    try:
        while True:
            current = inst.query('MEAS:CURR?')
            current_float = float(current.strip())
            if current_float > 0.001: # timer starts when current >0.001A
                for iteration in range(30):
                    current = inst.query('MEAS:CURR?')
                    current_float = float(current.strip())
                    current_values.append(current_float)
                    time.sleep(1)
                last_5_avg_current = sum(current_values[-5:]) / 5
                total_avg_current = sum(current_values)/len(current_values)
                return last_5_avg_current
            time.sleep(1)
    finally:
        inst.write('OUTP OFF')
        inst.close()
        print("Power Supply Connection Closed")

def psuedo_current(model_voltage):
    # Skip pyvisa connection and use pseudo measurement data
    # Simulate a random current between 0.5 and 1.0 A for testing GUI
    pseudo_current = random.uniform(0.5, 1.0)
    return pseudo_current

def get_pf_status(model, avg_current):
    criteria = MODEL_CRITERIA.get(model)
    if not criteria:
        # Default logic if model not found
        return "PASS" if avg_current > 0.8 else "FAIL(F7A)" if avg_current > 0.6 else "FAIL(F7P)"
    if model == "S931":
        if avg_current > criteria.get("PASS", 0.8):
            return "PASS"
        elif avg_current > criteria.get("F7A", 0.6):
            return "FAIL(F7A)"
        else:
            return "FAIL(F7P)"
    elif model == "S721":
        if avg_current > criteria.get("PASS", 0.75):
            return "PASS"
        elif avg_current > criteria.get("F7P", 0.6):
            return "FAIL(F7P)"
        else:
            return "FAIL(F7A)"
    # Add more models as needed
    # Default fallback
    return "PASS" if avg_current > 0.8 else "FAIL(F7A)" if avg_current > 0.6 else "FAIL(F7P)"

# --------- New: Background measurement with progress updates ---------
def measure_current_and_get_avg_with_progress(model_voltage, progress_queue, stop_event, prompt_response_queue=None, gpib_address=6):
    """
    Runs on a background thread. Sends structured messages to progress_queue:
      ("status", text)                     - general status text
      ("phase", "waiting"|"measuring")      - phase transitions
      ("tick", i, total)                   - sampling progress (i from 1..total)
      ("done", avg_last5, avg_total)       - success result
      ("error", error_message)             - error occurred
      ("prompt_device_check", question)    - ask operator about device connection
      ("sub_pba_fail",)                    - user confirmed device but no current
    stop_event can be used to cancel.
    prompt_response_queue carries boolean answers from the UI prompts.
    """
    if USE_PSEUDO_CURRENT:
        try:
            progress_queue.put(("status", "Using pseudo current mode (power supply not required)."))
            progress_queue.put(("phase", "waiting"))
            progress_queue.put(("status", "Simulating device connection..."))
            for _ in range(5):
                if stop_event.is_set():
                    raise Exception("Measurement cancelled by user.")
                time.sleep(0.2)

            total_samples = 30
            current_values = []
            progress_queue.put(("phase", "measuring"))
            progress_queue.put(("status", "Simulating current measurements"))

            for i in range(total_samples):
                if stop_event.is_set():
                    raise Exception("Measurement cancelled by user.")
                current_value = psuedo_current(model_voltage)
                current_values.append(current_value)
                progress_queue.put(("tick", i + 1, total_samples))
                for _ in range(5):
                    if stop_event.is_set():
                        raise Exception("Measurement cancelled by user.")
                    time.sleep(0.04)

            last_5_avg_current = sum(current_values[-5:]) / 5 if len(current_values) >= 5 else sum(current_values)/len(current_values)
            total_avg_current = sum(current_values)/len(current_values)
            progress_queue.put(("done", last_5_avg_current, total_avg_current))
        except Exception as e:
            progress_queue.put(("error", str(e)))
        return

    resource_str = f'GPIB0::{gpib_address}::INSTR'
    inst = None
    try:
        progress_queue.put(("status", f"Connecting to power supply at GPIB address {gpib_address}..."))
        rm = pyvisa.ResourceManager()
        inst = rm.open_resource(resource_str)
        inst.write('OUTP ON')
        inst.write(f"VOLT {model_voltage}")

        current_values = []

        progress_queue.put(("phase", "waiting"))
        progress_queue.put(("status", "Plug Anyway Jig into the phone."))
        zero_current_start = None
        while True:
            if stop_event.is_set():
                raise Exception("Measurement cancelled by user.")
            current = inst.query('MEAS:CURR?')
            current_float = float(current.strip())
            if current_float > 0.001:
                break
            if zero_current_start is None:
                zero_current_start = time.time()
            elif time.time() - zero_current_start >= 10:
                if prompt_response_queue is None:
                    progress_queue.put(("error", "Unable to confirm device connection. Please retry."))
                    return
                progress_queue.put(("status", "No current detected for 10 seconds."))
                progress_queue.put(("prompt_device_check", "No current detected for 10 seconds. Is a device connected to the power supply?"))
                while True:
                    if stop_event.is_set():
                        raise Exception("Measurement cancelled by user.")
                    try:
                        user_answer = prompt_response_queue.get(timeout=0.1)
                        break
                    except queue.Empty:
                        continue
                if user_answer:
                    progress_queue.put(("sub_pba_fail",))
                    return
                zero_current_start = time.time()
            time.sleep(0.2)  # poll faster to react quickly

        total_samples = 30
        progress_queue.put(("phase", "measuring"))
        progress_queue.put(("status", "Measuring current"))

        for i in range(total_samples):
            if stop_event.is_set():
                raise Exception("Measurement cancelled by user.")
            current = inst.query('MEAS:CURR?')
            current_float = float(current.strip())
            current_values.append(current_float)
            progress_queue.put(("tick", i + 1, total_samples))
            time.sleep(1)

        last_5_avg_current = sum(current_values[-5:]) / 5 if len(current_values) >= 5 else sum(current_values)/len(current_values)
        total_avg_current = sum(current_values)/len(current_values)
        progress_queue.put(("done", last_5_avg_current, total_avg_current))
    except Exception as e:
        progress_queue.put(("error", str(e)))
    finally:
        try:
            if inst is not None:
                inst.write('OUTP OFF')
                inst.close()
        except Exception:
            pass

# --------- Login Window ---------
current_user = {"username": "", "is_admin": False}
username_list = ["admin", "worker1", "worker2", "worker3", "worker4"]
password_list = {"admin":"", "worker1": "1", "worker2": "2","worker3": "3","worker4": "4"}

def login():
    username = entry_username.get()
    password = entry_password.get()
    
    if username in username_list and password == password_list.get(username):
        # admin cell editing privilege
        if username == "admin":
            current_user["username"] = username
            current_user["is_admin"] = True
            messagebox.showinfo("Login", "Login successful!")
            root.withdraw()
            open_main_window()
        else:
            current_user["username"] = username
            current_user["is_admin"] = False
            messagebox.showinfo("Login", "Login successful!")
            root.withdraw()
            open_main_window()
    else:
        messagebox.showerror("Login", "Invalid credentials.")

root = tk.Tk()
root.title("AutoPowerTest Login")

tk.Label(root, text="Username:", font=("TkDefaultFont", 26)).grid(row=0, column=0, padx=50, pady=50)
entry_username = tk.Entry(root, font=("TkDefaultFont", 26))
entry_username.grid(row=0, column=1, padx=30, pady=30)

tk.Label(root, text="Password:", font=("TkDefaultFont", 26)).grid(row=1, column=0, padx=50, pady=50)
entry_password = tk.Entry(root, show="*", font=("TkDefaultFont", 26))
entry_password.grid(row=1, column=1, padx=30, pady=30)

tk.Button(root, text="Login", font=("TkDefaultFont", 20), command=login).grid(row=2, column=0, columnspan=2, padx=10, pady=10)

def open_main_window():
    main_window = tk.Toplevel()
    main_window.title("Test Result")
    main_window.geometry("1200x700")

    columns = ("IMEI", "Model", "Avg. Current", "P/F", "Worker ID", "Date")
    tree = ttk.Treeview(main_window, columns=columns, show="headings", selectmode="extended")
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=120, anchor='center')

    tree.tag_configure('green_row', background="#b6fcd5")
    tree.tag_configure('red_row', background="#ffb6b6")

    scrollbar = ttk.Scrollbar(main_window, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=scrollbar.set)

    tree.grid(row=0, column=0, sticky="nsew")
    scrollbar.grid(row=0, column=1, sticky="ns")
    main_window.grid_rowconfigure(0, weight=1)
    main_window.grid_columnconfigure(0, weight=1)

    pseudo_mode_var = tk.BooleanVar(value=USE_PSEUDO_CURRENT)

    def on_toggle_pseudo_mode():
        global USE_PSEUDO_CURRENT
        USE_PSEUDO_CURRENT = pseudo_mode_var.get()

    pseudo_check = tk.Checkbutton(
        main_window,
        text="Use pseudo current (no power supply)",
        font=("TkDefaultFont", 12),
        variable=pseudo_mode_var,
        command=on_toggle_pseudo_mode
    )
    pseudo_check.grid(row=2, column=0, columnspan=2, pady=5)

    # --------- Model settings editor (Admin only) ---------
    def open_model_settings_dialog(parent):
        if not current_user["is_admin"]:
            messagebox.showinfo("Permission denied", "Only admin can edit model settings.")
            return

        dialog = tk.Toplevel(parent)
        dialog.title("Model Settings (Admin Only)")
        dialog.geometry("1000x520")
        dialog.grab_set()
        dialog.grid_columnconfigure(0, weight=0)
        dialog.grid_columnconfigure(1, weight=0)
        dialog.grid_columnconfigure(2, weight=1)
        dialog.grid_rowconfigure(1, weight=1)
        dialog.grid_rowconfigure(2, weight=1)


        # Model name
        tk.Label(dialog, text="Model:", font=("TkDefaultFont", 14))\
            .grid(row=0, column=0, padx=10, pady=10, sticky="e")
        model_var = tk.StringVar()
        model_combo_settings = ttk.Combobox(
            dialog, textvariable=model_var,
            values=sorted(MODEL_VOLTAGE_MAP.keys()),
            state="readonly",
            font=("TkDefaultFont", 14)
        )
        model_combo_settings.grid(row=0, column=1, padx=10, pady=10, sticky="w")

        # New model name entry (for adding)
        tk.Label(dialog, text="(Or new model name):", font=("TkDefaultFont", 10))\
            .grid(row=1, column=0, padx=10, pady=0, sticky="e")
        new_model_entry = tk.Entry(dialog, font=("TkDefaultFont", 12))
        new_model_entry.grid(row=1, column=1, padx=10, pady=0, sticky="w")

        # Voltage entry
        tk.Label(dialog, text="Boot On Voltage (V):", font=("TkDefaultFont", 14))\
            .grid(row=2, column=0, padx=10, pady=10, sticky="e")
        voltage_entry = tk.Entry(dialog, font=("TkDefaultFont", 14))
        voltage_entry.grid(row=2, column=1, padx=10, pady=10, sticky="w")

        # GPIB Address entry
        tk.Label(dialog, text="GPIB Address:", font=("TkDefaultFont", 14))\
            .grid(row=3, column=0, padx=10, pady=10, sticky="e")
        gpib_address_entry = tk.Entry(dialog, font=("TkDefaultFont", 14))
        gpib_address_entry.grid(row=3, column=1, padx=10, pady=10, sticky="w")

        # Criteria editor
        tk.Label(dialog, text="Criteria (KEY=VALUE, one per line):", font=("TkDefaultFont", 14))\
            .grid(row=4, column=0, padx=10, pady=10, sticky="ne")
        criteria_text = tk.Text(dialog, width=40, height=10, font=("TkDefaultFont", 12))
        criteria_text.grid(row=4, column=1, padx=10, pady=10, sticky="w")
        # Current settings overview
        tk.Label(dialog, text="Model / Boot On Voltage / GPIB Address / Criteria", font=("TkDefaultFont", 12, "bold"))\
            .grid(row=0, column=2, padx=(30, 10), pady=(10, 5), sticky="nw")
        current_values_text = tk.Text(dialog, width=70, height=16, font=("TkDefaultFont", 11), state="disabled")
        current_values_text.grid(row=1, column=2, rowspan=5, padx=(30, 10), pady=(0, 10), sticky="nsew")

        def format_criteria_for_display(criteria_dict):
            if not criteria_dict:
                return "No criteria set"
            parts = []
            for key in sorted(criteria_dict.keys()):
                parts.append(f"{key}={criteria_dict[key]:g}")
            return ", ".join(parts)

        def refresh_current_values_display():
            lines = []
            for model_name in sorted(MODEL_VOLTAGE_MAP.keys()):
                voltage_value = MODEL_VOLTAGE_MAP.get(model_name, "")
                gpib_addr = MODEL_GPIB_ADDRESS.get(model_name, 6)
                criteria_value = MODEL_CRITERIA.get(model_name, {})
                criteria_str = format_criteria_for_display(criteria_value)
                lines.append(f"{model_name} / {voltage_value}V / GPIB:{gpib_addr} / {criteria_str}")
            text_output = "\n".join(lines) if lines else "No models configured."
            current_values_text.config(state="normal")
            current_values_text.delete("1.0", tk.END)
            current_values_text.insert(tk.END, text_output)
            current_values_text.config(state="disabled")

        refresh_current_values_display()


        def load_model_settings(*args):
            model = model_var.get()
            if not model:
                return
            # Clear new model field when selecting existing
            new_model_entry.delete(0, tk.END)

            # Load voltage
            voltage = MODEL_VOLTAGE_MAP.get(model, "")
            voltage_entry.delete(0, tk.END)
            voltage_entry.insert(0, str(voltage))

            # Load GPIB address
            gpib_addr = MODEL_GPIB_ADDRESS.get(model, 6)
            gpib_address_entry.delete(0, tk.END)
            gpib_address_entry.insert(0, str(gpib_addr))

            # Load criteria
            criteria = MODEL_CRITERIA.get(model, {})
            criteria_text.delete("1.0", tk.END)
            for k, v in criteria.items():
                criteria_text.insert(tk.END, f"{k}={v}\n")

        model_combo_settings.bind("<<ComboboxSelected>>", load_model_settings)

        # Preselect first model if available
        if MODEL_VOLTAGE_MAP:
            first_model = sorted(MODEL_VOLTAGE_MAP.keys())[0]
            model_var.set(first_model)
            load_model_settings()

        def save_model_settings():
            # Decide model name (existing or new)
            model = new_model_entry.get().strip() or model_var.get().strip()
            if not model:
                messagebox.showwarning("Model Settings", "Please select or enter a model name.")
                return

            # Parse voltage
            try:
                v = float(voltage_entry.get().strip())
            except ValueError:
                messagebox.showwarning("Model Settings", "Voltage must be a number.")
                return

            # Parse GPIB address
            try:
                gpib_addr = int(gpib_address_entry.get().strip())
                if gpib_addr < 0 or gpib_addr > 30:
                    messagebox.showwarning("Model Settings", "GPIB address must be between 0 and 30.")
                    return
            except ValueError:
                messagebox.showwarning("Model Settings", "GPIB address must be an integer.")
                return

            # Parse criteria lines: KEY=VALUE
            new_criteria = {}
            for line in criteria_text.get("1.0", tk.END).splitlines():
                line = line.strip()
                if not line:
                    continue
                if "=" not in line:
                    messagebox.showwarning("Model Settings", f"Invalid criteria line: {line}")
                    return
                key, val = line.split("=", 1)
                key = key.strip()
                try:
                    val = float(val.strip())
                except ValueError:
                    messagebox.showwarning("Model Settings", f"Invalid value in line: {line}")
                    return
                new_criteria[key] = val

            # Update dictionaries
            MODEL_VOLTAGE_MAP[model] = v
            MODEL_CRITERIA[model] = new_criteria
            MODEL_GPIB_ADDRESS[model] = gpib_addr

            # Persist to config file
            try:
                save_config()
            except Exception as e:
                messagebox.showerror("Model Settings", f"Failed to save config:\n{e}")
                return

            # Refresh comboboxes that use model list
            try:
                model_combo["values"] = list(MODEL_VOLTAGE_MAP.keys())
            except Exception:
                pass
            model_combo_settings["values"] = sorted(MODEL_VOLTAGE_MAP.keys())
            model_var.set(model)
            refresh_current_values_display()

            messagebox.showinfo("Model Settings", "Model settings saved.")

        save_btn = tk.Button(dialog, text="Save", font=("TkDefaultFont", 14),
                             command=save_model_settings)
        save_btn.grid(row=5, column=0, columnspan=2, pady=10)

    # --------- Add Row Dialog ---------
    def add_row_dialog():
        dialog = tk.Toplevel(main_window)
        dialog.title("Add New Measurement")
        dialog.grab_set()
        
        tk.Label(dialog, text="IMEI:", font=("TkDefaultFont", 20)).grid(row=0, column=0, padx=10, pady=10)
        imei_entry = tk.Entry(dialog,  font=("TkDefaultFont", 20))
        imei_entry.grid(row=0, column=1, padx=10, pady=10)

        tk.Label(dialog, text="Model:", font=("TkDefaultFont", 20)).grid(row=1, column=0, padx=10, pady=10)
        model_var = tk.StringVar()
        # model_combo must be accessible in settings dialog
        nonlocal_model_combo_container = {}

        model_combo = ttk.Combobox(
            dialog, textvariable=model_var,
            values=list(MODEL_VOLTAGE_MAP.keys()),
            state="readonly",  font=("TkDefaultFont", 20)
        )
        model_combo.grid(row=1, column=1, padx=10, pady=10)
        if MODEL_VOLTAGE_MAP:
            model_combo.current(0)  # Set default selection

        # expose model_combo to model settings dialog via closure
        nonlocal_model_combo_container["combo"] = model_combo

        def is_valid_imei(imei):
            return imei.isdigit() and len(imei) == 15 and imei.startswith("3")

        def run_measurement():
            global ACTIVE_MEASUREMENT
            imei = imei_entry.get().strip()
            model = model_var.get().strip()
            if not is_valid_imei(imei) or not model:
                messagebox.showwarning("Input Error", "Please enter correct IMEI and Model.")
                return

            # Hide entry dialog while measuring
            dialog.withdraw()

            # Progress dialog
            progress_dialog = tk.Toplevel(main_window)
            progress_dialog.title("Measuring...")
            progress_dialog.transient(main_window)
            progress_dialog.grab_set()

            status_label = tk.Label(progress_dialog, text="Initializing...", font=("TkDefaultFont", 14))
            status_label.grid(row=0, column=0, columnspan=2, padx=15, pady=(15, 5))

            # Spinner initially (indeterminate)
            pb = ttk.Progressbar(progress_dialog, mode='indeterminate', length=300)
            pb.grid(row=1, column=0, columnspan=2, padx=15, pady=10)
            pb.start(10)

            timer_label = tk.Label(progress_dialog, text="", font=("TkDefaultFont", 12))
            timer_label.grid(row=2, column=0, columnspan=2, padx=15, pady=(0, 10))

            cancel_btn = tk.Button(progress_dialog, text="Cancel", font=("TkDefaultFont", 12))
            cancel_btn.grid(row=3, column=0, columnspan=2, padx=15, pady=(0, 15))

            # Communication with worker thread
            q = queue.Queue()
            prompt_response_queue = queue.Queue()
            stop_event = threading.Event()

            def on_cancel():
                cancel_btn.config(state="disabled")
                stop_event.set()
                status_label.config(text="Cancelling...")

            cancel_btn.config(command=on_cancel)

            # Start background measurement
            model_voltage = MODEL_VOLTAGE_MAP.get(model, 5.8)
            gpib_address = MODEL_GPIB_ADDRESS.get(model, 6)
            worker = threading.Thread(
                target=measure_current_and_get_avg_with_progress,
                args=(model_voltage, q, stop_event, prompt_response_queue, gpib_address),
                daemon=True
            )
            worker.start()
            ACTIVE_MEASUREMENT["thread"] = worker
            ACTIVE_MEASUREMENT["stop_event"] = stop_event

            # State for progress phase
            sampling_total = 30
            in_sampling = False
            sampling_start_ts = None

            def poll_queue():
                nonlocal in_sampling, sampling_start_ts

                try:
                    while True:
                        msg = q.get_nowait()
                        kind = msg[0]

                        if kind == "status":
                            status_label.config(text=msg[1])

                        elif kind == "phase":
                            phase = msg[1]
                            if phase == "waiting":
                                # Indeterminate spinner while waiting for threshold
                                if pb['mode'] != 'indeterminate':
                                    pb.stop()
                                    pb.config(mode='indeterminate', maximum=100, value=0)
                                    pb.start(10)
                                timer_label.config(text="Waiting for device current...")
                                in_sampling = False
                                sampling_start_ts = None
                            elif phase == "measuring":
                                # Switch to determinate 30-second sampling
                                pb.stop()
                                pb.config(mode='determinate', maximum=sampling_total, value=0, length=300)
                                in_sampling = True
                                sampling_start_ts = time.time()
                                timer_label.config(text=f"Remaining: {sampling_total}s")

                        elif kind == "tick":
                            i, total = msg[1], msg[2]
                            pb['value'] = i
                            remaining = max(total - i, 0)
                            timer_label.config(text=f"Remaining: {remaining}s")

                        elif kind == "prompt_device_check":
                            question = msg[1] if len(msg) > 1 else "Is a device connected to the power supply?"
                            answer = messagebox.askyesno("Confirm Device Connection", question)
                            prompt_response_queue.put(answer)

                        elif kind == "done":
                            avg_last5, _avg_total = msg[1], msg[2]
                            # Close progress dialog and finish up
                            progress_dialog.grab_release()
                            progress_dialog.destroy()

                            try:
                                avg_current = float(avg_last5)
                                pf_status = get_pf_status(model, avg_current)
                                id_val = current_user["username"]
                                date_val = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                row = (imei, model, f"{avg_current:.3f}"+"A", pf_status, id_val, date_val)
                                tag = 'green_row' if pf_status == 'PASS' else 'red_row'
                                tree.insert("", tk.END, values=row, tags=(tag,))
                                messagebox.showinfo("Measurement", pf_status)

                                # --- Append to Excel log file ---
                                try:
                                    if os.path.exists(log_saved_path):
                                        old_df = pd.read_excel(log_saved_path)
                                    else:
                                        old_df = pd.DataFrame(columns=columns)
                                except Exception:
                                    old_df = pd.DataFrame(columns=columns)

                                new_df = pd.DataFrame([row], columns=columns)
                                combined_df = pd.concat([old_df, new_df], ignore_index=True)
                                combined_df.to_excel(log_saved_path, index=False)
                            except Exception as e:
                                messagebox.showerror("Measurement Error", f"Failed to process result: {e}")
                            finally:
                                dialog.destroy()
                                ACTIVE_MEASUREMENT["thread"] = None
                                ACTIVE_MEASUREMENT["stop_event"] = None
                            return  # stop polling

                        elif kind == "sub_pba_fail":
                            progress_dialog.grab_release()
                            progress_dialog.destroy()
                            try:
                                pf_status = "FAIL(Sub PBA)"
                                avg_current = "0.000A"
                                id_val = current_user["username"]
                                date_val = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                row = (imei, model, avg_current, pf_status, id_val, date_val)
                                tree.insert("", tk.END, values=row, tags=('red_row',))
                                messagebox.showinfo("Measurement", pf_status)
                                try:
                                    if os.path.exists(log_saved_path):
                                        old_df = pd.read_excel(log_saved_path)
                                    else:
                                        old_df = pd.DataFrame(columns=columns)
                                except Exception:
                                    old_df = pd.DataFrame(columns=columns)
                                new_df = pd.DataFrame([row], columns=columns)
                                combined_df = pd.concat([old_df, new_df], ignore_index=True)
                                combined_df.to_excel(log_saved_path, index=False)
                            except Exception as e:
                                messagebox.showerror("Measurement Error", f"Failed to log Sub PBA failure: {e}")
                            finally:
                                dialog.destroy()
                                ACTIVE_MEASUREMENT["thread"] = None
                                ACTIVE_MEASUREMENT["stop_event"] = None
                            return  # stop polling

                        elif kind == "error":
                            err = msg[1]
                            progress_dialog.grab_release()
                            progress_dialog.destroy()
                            messagebox.showerror("Measurement Error", f"{err}")
                            dialog.destroy()
                            ACTIVE_MEASUREMENT["thread"] = None
                            ACTIVE_MEASUREMENT["stop_event"] = None
                            return  # stop polling
                except queue.Empty:
                    pass

                # Continue polling
                progress_dialog.after(100, poll_queue)

            poll_queue()

        tk.Button(dialog, text="Run Measurement", command=run_measurement).grid(row=2, column=0, columnspan=2, pady=10)

        # Make model_combo available to settings dialog via closure
        def patch_settings_dialog_combo():
            # Assign the actual combobox into enclosing scope used by open_model_settings_dialog
            nonlocal_model_combo_container["combo_ref"] = model_combo

        patch_settings_dialog_combo()

    add_row_btn = tk.Button(main_window, text="Add New Measurement", command=add_row_dialog, font=("TkDefaultFont", 15))
    add_row_btn.grid(row=3, column=0, columnspan=2, pady=10)

    # --------- Delete Selected Rows ---------
    def delete_selected_rows():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Delete Row", "Please select one or more rows to delete.")
            return
        for item in selected:
            tree.delete(item)

    # --------- Hide Delete Button for Non-Admin ---------
    if current_user["is_admin"]:
        delete_btn = tk.Button(main_window, text="Delete Selected Row(s)", command=delete_selected_rows, font=("TkDefaultFont", 15))
        delete_btn.grid(row=4, column=0, columnspan=2, pady=10)

    # --------- Export to Excel ---------
    def export_to_excel():
        rows = [tree.item(item)['values'] for item in tree.get_children()]
        if not rows:
            messagebox.showwarning("Export Data", "No data to export.")
            return
        df = pd.DataFrame(rows, columns=columns)
        file_path = filedialog.asksaveasfilename(defaultextension='.xlsx', filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            try:
                df.to_excel(file_path, index=False)
                messagebox.showinfo("Export Data", f"Data exported successfully to {file_path}")
            except Exception as e:
                messagebox.showerror("Export Error", f"Failed to export data.\n{e}")

    export_btn = tk.Button(main_window, text="Export to Excel", command=export_to_excel, font=("TkDefaultFont", 15))
    export_btn.grid(row=5, column=0, columnspan=2, pady=10)

    # --------- Model Settings Button (Admin only) ---------
    if current_user["is_admin"]:
        settings_btn = tk.Button(
            main_window,
            text="Edit Model Settings",
            font=("TkDefaultFont", 15),
            command=lambda: open_model_settings_dialog(main_window)
        )
        settings_btn.grid(row=6, column=0, columnspan=2, pady=10)

    # --------- Cell Editing (Admin Only) ---------
    edit_box = None
    def on_double_click(event):
        nonlocal edit_box

        if not current_user["is_admin"]:
            messagebox.showinfo("Permission denied", "Only admin can edit cells.")
            return

        region = tree.identify("region", event.x, event.y)
        if region != "cell":
            return
        row_id = tree.identify_row(event.y)
        col_id = tree.identify_column(event.x)
        col_index = int(col_id.replace("#", "")) - 1

        if not row_id or col_index < 0:
            return

        bbox = tree.bbox(row_id, col_id)
        if not bbox:
            return
        x, y, width, height = bbox

        values = list(tree.item(row_id, "values"))
        current_value = values[col_index]

        if edit_box:
            edit_box.destroy()
        edit_box = tk.Entry(main_window)
        edit_box.place(x=x, y=y, width=width, height=height)
        edit_box.insert(0, current_value)
        edit_box.focus_set()

        def save_new_value(event=None):
            new_value = edit_box.get()
            values[col_index] = new_value
            if columns[col_index] == "P/F":
                tag = 'green_row' if new_value == 'PASS' else 'red_row' if new_value.startswith('FAIL') else ''
                tree.item(row_id, values=values, tags=(tag,) if tag else ())
            else:
                tree.item(row_id, values=values)
            edit_box.destroy()

        edit_box.bind("<Return>", save_new_value)
        edit_box.bind("<FocusOut>", save_new_value)

    tree.bind("<Double-1>", on_double_click)

    def on_close():
        global ACTIVE_MEASUREMENT
        stop_event = ACTIVE_MEASUREMENT.get("stop_event")
        worker = ACTIVE_MEASUREMENT.get("thread")
        if stop_event is not None:
            stop_event.set()
        if worker is not None and worker.is_alive():
            worker.join(timeout=2)
        ACTIVE_MEASUREMENT["thread"] = None
        ACTIVE_MEASUREMENT["stop_event"] = None
        main_window.destroy()
        root.destroy()
    main_window.protocol("WM_DELETE_WINDOW", on_close)

root.mainloop()
